In [2]:
import sys

sys.path.append("..")

from itertools import product
import pandas as pd
import kornia.augmentation as K
import torch
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from src.models import get_model_by_name
from src.datasets.eurosat import EuroSATMinimal
from src.utils import extract_features
from tqdm import tqdm

device = torch.device("cuda:1")

In [3]:
# Fit and save features
model_names = [
    "resnet50_pretrained_seco",
    "resnet50_pretrained_moco",
    "resnet50_pretrained_imagenet",
    "resnet50_randominit",
    "imagestats",
    "mosaiks_512_3",
    "mosaiks_zca_512_3",
]
rgbs = [True, False]
sizes = [64, 224]
normalization_methods = ["divide", "standardization", "for_seco"]
use_both_train_val = [True, False]

columns = {
    "model_name": [],
    "rgb": [],
    "size": [],
    "normalization": [],
    "use_train_val": [],
    "accuracy": [],
    "accuracy_scaler": [],
}

for model_name, rgb, size, normalization, use_both in tqdm(product(model_names, rgbs, sizes, normalization_methods, use_both_train_val), total=6*2*2*2*2):
    transforms = nn.Sequential(K.Resize(size)).to(device)
    dm = EuroSATMinimal(
        root="../data/eurosat/",
        band_set="rgb" if rgb else "all",
        normalization_method=normalization,
        batch_size=64,
        num_workers=8,
        use_both_trainval=use_both,
    )
    dm.setup()

    if "seco" in model_name and not rgb:
        continue

    model = get_model_by_name(model_name, rgb, device=device, dataset=dm.train_dataset)

    x_train, y_train = extract_features(
        model, dm.train_dataloader(), device, transforms=transforms, verbose=False
    )
    x_test, y_test = extract_features(
        model, dm.test_dataloader(), device, transforms=transforms, verbose=False
    )

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc = knn_model.score(x_test, y_test)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc_scaler = knn_model.score(x_test, y_test)

    columns["model_name"].append(model_name)
    columns["rgb"].append(rgb)
    columns["size"].append(size)
    columns["normalization"].append(normalization)
    columns["use_train_val"].append(use_both)
    columns["accuracy"].append(acc)
    columns["accuracy_scaler"].append(acc_scaler)

168it [1:06:52, 23.89s/it]                     


In [4]:
df = pd.DataFrame(columns)

In [5]:
df.to_csv("../results/eurosat_table_results.csv")